In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Endometrioid_Cancer/GSE109059'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Blocking and randomization to improve molecular biomarker discovery"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['disease state: advanced serous ovarian cancer', 'disease state: endometrioid endometrial cancer'], 1: ['tissue: tumor'], 2: [nan, 'batch: BATCH1', 'batch: BATCH2', 'batch: BATCH3', 'batch: BATCH4', 'batch: BATCH5']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

import numpy as np  # Add this import statement

# Define Sample_Characteristics_Dict from the provided data
Sample_Characteristics_Dict = {
    0: ['disease state: advanced serous ovarian cancer', 'disease state: endometrioid endometrial cancer'],
    1: ['tissue: tumor'],
    2: [np.nan, 'batch: BATCH1', 'batch: BATCH2', 'batch: BATCH3', 'batch: BATCH4', 'batch: BATCH5']
}

# 1. Gene Expression Data Availability
is_gene_available = True  # Since the data seems to include gene expression data

# 2. Variable Availability and Data Type Conversion
# Based on the provided characteristics dictionary:

# For Endometrioid_Cancer
if 0 in Sample_Characteristics_Dict and 'endometrioid endometrial cancer' in Sample_Characteristics_Dict[0]:
    trait_row = 0

# For age (None in this dataset, as there is no available data related to age directly)
age_row = None

# For gender (None in this dataset, as there is no available data related to gender directly)
gender_row = None

# 2.3 Data Type Conversion Functions
def convert_trait(value):
    # Assuming 'endometrioid endometrial cancer' is the presence of the trait and converting it to 1, others to 0
    val = value.split(":")[1].strip().lower() if ":" in value else None
    if val == 'endometrioid endometrial cancer':
        return 1
    elif val == 'advanced serous ovarian cancer':
        return 0
    else:
        return None

def convert_age(value):
    return None  # Age data not available

def convert_gender(value):
    return None  # Gender data not available

# 3. Save Metadata
save_cohort_info('GSE109059', './preprocessed/Endometrioid_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction (Skip if trait_row is None)
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Endometrioid_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Endometrioid_Cancer/trait_data/GSE109059.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
